# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from scipy import stats as st
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.preprocessing import OrdinalEncoder 
import warnings
warnings.filterwarnings("ignore")


In [2]:
geo_data_1 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_0.csv')
geo_data_2 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_1.csv')
geo_data_3 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_2.csv')

- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).

Рассмотрим каждый датасет отдельно.

In [3]:
# Первый датасет geo_data_1
geo_data_1.info()
print(geo_data_1.describe())
geo_data_1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.343769   

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
# Второй датасет geo_data_2
geo_data_2.info()
print(geo_data_2.describe())
geo_data_2.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144       0.000000
25%        -6.298551      -8.267985       1.000021      26.953261
50%         1.153055      -4.813172       2.011479      57.085625
75%         8.621015      -1.332816       3.999904     107.813044
max        29.421755      18.734063   

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [5]:
# Третий датасет geo_data_3
geo_data_3.info()
print(geo_data_3.describe())
geo_data_3.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.002023      -0.002081       2.495128      95.000000
std         1.732045       1.730417       3.473445      44.749921
min        -8.760004      -7.084020     -11.970335       0.000000
25%        -1.162288      -1.174820       0.130359      59.450441
50%         0.009424      -0.009482       2.484236      94.925613
75%         1.158535       1.163678       4.858794     130.595027
max         7.238262       7.844801   

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [6]:
print(geo_data_1.duplicated().sum())
geo_data_1.isna().sum()


0


id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [7]:
print(geo_data_2.duplicated().sum())
geo_data_2.isna().sum()

0


id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [8]:
print(geo_data_3.duplicated().sum())
geo_data_3.isna().sum()

0


id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

Дупликатов и нулевых значений нет во всех датасетах

В третьем датасете средняя mean выше, чем в остальных датасетах. <br>
Уверен, не всё так просто и легко.<br>
Также, во втором датасете можно заметить показатель mean сильно ниже, чем в 1 и 3 датасете. <br>
Однако значения все еще непонятны и не представляют ничего ценного на первый взгляд, будет правильным посмотреть на зависимость столбцов f0,f1,f2 от значения product

In [9]:
#Зависимость объема от признаков для geo_data1
corr1 = geo_data_1.corr()
corr1['product']

f0         0.143536
f1        -0.192356
f2         0.483663
product    1.000000
Name: product, dtype: float64

In [10]:
#Зависимость объема от признаков для geo_data2
corr2 = geo_data_2.corr()
corr2['product']

f0        -0.030491
f1        -0.010155
f2         0.999397
product    1.000000
Name: product, dtype: float64

In [11]:
#Зависимость объема от признаков для geo_data3
corr3 = geo_data_3.corr()
corr3['product']

f0        -0.001987
f1        -0.001012
f2         0.445871
product    1.000000
Name: product, dtype: float64

Сильно очень бросается в глаза, что в датасете geo_data2 невероятно большая зависимость значений product от f2 показателя. <br>
Это может сильно повлиять на результаты моделей, но пока не ясно положительно или отрицательно

А в целом же, зависимость f2 для значений product у всех датасетов сравнительно f0,f1 сильно выше. 


## Обучение и проверка модели

In [12]:
def split_train(data):
    """
    Функция берет данные региона.
    Подготавливает данные fearures и train для модели.
    Обучает модель линейной регрессии
    Возвращает среднеквадратичную ошибку и средний объем нефти в скважинах этого региона.
    """
    
    features = data.drop(["product","id"], axis=1)
    target = data["product"]
    features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                                  target, 
                                                                                  test_size=0.25, 
                                                                                  random_state= 1)
    scaler = MinMaxScaler()
    features_train = scaler.fit_transform(features_train)
    features_valid = scaler.transform(features_valid)
    
    lr = LinearRegression(normalize=False)
    lr.fit(features_train, target_train)
    predictions = lr.predict(features_valid)
    predictions = pd.Series(predictions)
    rmse = (mean_squared_error(target_valid, predictions))**(0.5)
    r2 =  r2_score(target_valid, predictions)
    average_product = sum(predictions) / len(predictions)
    print("RMSE: {0:.2f}".format(rmse))
    print("r2: {0:.2f}".format(r2))
    print("Average product: {0:.2f}".format(average_product))
    
    return (predictions, target_valid.reset_index(drop=True), rmse)

In [13]:
#geo_data_1
print(f"Регион 1:")
print()
split_train(geo_data_1)
print()

Регион 1:

RMSE: 37.74
r2: 0.28
Average product: 92.49



In [14]:
#geo_data_2
print(f"Регион 2:")
print()
split_train(geo_data_2)
print()

Регион 2:

RMSE: 0.89
r2: 1.00
Average product: 69.12



In [15]:
#geo_data_3
print(f"Регион 3:")
print()
split_train(geo_data_3)
print()

Регион 3:

RMSE: 39.87
r2: 0.20
Average product: 94.96



In [16]:
#сохраним все важные значения
predictions_1, validation_1, rmse_1 = split_train(geo_data_1)
predictions_2, validation_2, rmse_2 = split_train(geo_data_2)
predictions_3, validation_3, rmse_3 = split_train(geo_data_3)

RMSE: 37.74
r2: 0.28
Average product: 92.49
RMSE: 0.89
r2: 1.00
Average product: 69.12
RMSE: 39.87
r2: 0.20
Average product: 94.96


Самая минимальная среднеквадратичная ошибка у Региона 2. RMSE: 0.89<br> 
Также, у этого региона r2 score равен единице. <br>
Думаю, это связано с тем, что очень большая зависимость была у признака f2 к product.<br>
Средний запас объема нефти в этом регионе - 68.73 тыс.баррелей

## Подготовка к расчёту прибыли

Бюджет на разработку скважин в регионе — 10 млрд рублей. <br>
При нынешних ценах один баррель сырья приносит 450 рублей дохода. <br>
Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

In [1]:
BUDGET = 10**10 
INCOME_PER_BARREL = 450 * 10**3 

Точка безубыточности — размер выручки, при котором бизнес работает «в ноль»: его доходы равны расходам. Всё, что бизнес зарабатывает сверх значения в точке безубыточности, становится его прибылью.

Находится это значение по формуле:<br> 
$\Huge  x = {budget \over income}$

In [2]:
CVP = BUDGET / INCOME_PER_BARREL
print('Для работы "в ноль" необходимо  {0:.0f} тыс.баррелей'.format(CVP))

Для работы "в ноль" необходимо  22222 тыс.баррелей


In [19]:
print("Средний объем нефти в скважине для безубыточного функционирования: {0:.0f} тыс.баррелей".format(CVP/200))

Средний объем нефти в скважине для безубыточного функционирования: 111 тыс.баррелей


Объема нефти в скважинах значительно меньше, чем точка безубыточности. 

## Расчёт прибыли и рисков 

In [20]:
state = np.random.RandomState(12345)

In [21]:
EXPL = 500

In [22]:
count_of_wells = 200

In [23]:
TOP = 200

In [24]:
#функция для расчета прибыли с 200 лучших скважин
def profit(predictions, target):
    top_preds = predictions.sort_values(ascending=False)
    top_target = target[top_preds.index][:TOP]
    revenue = top_target.sum() * income_per_barrel
    return revenue - budget

In [25]:
def conf_interval_and_loss_risk(predictions, target):
    values = []
    for i in range(1000):
        target_sample = target.sample(EXPL, replace=True, random_state=state)
        predictions_sample = predictions[target_sample.index]
        values.append(profit(predictions_sample, target_sample))
        
    valuess = pd.Series(values)
    lower = valuess.quantile(0.025)
    higher = valuess.quantile(0.975)
    mean_revenue = valuess.mean()
    confidence_interval = (lower, higher)
    risk = st.percentileofscore(values, 0)
    return (confidence_interval, mean_revenue, risk)

In [26]:
reg = 1
for preds, target in zip([predictions_1, predictions_2, predictions_3], 
                         [validation_1, validation_2, validation_3]):
    interval, mean_revenue, risk = conf_interval_and_loss_risk(preds, target)
    print(f"Регион {reg}:")
    print("Средняя прибыль: {0:.2f}".format(mean_revenue))
    print("95% доверительный интервал: {}".format(interval))
    print("Риск убытков: {0:.2f}".format(risk))
    print()
    reg+=1

Регион 1:
Средняя прибыль: 458100401.80
95% доверительный интервал: (-91938453.32589239, 1015824888.6825125)
Риск убытков: 4.60

Регион 2:
Средняя прибыль: 533056709.60
95% доверительный интервал: (99430448.40112402, 973064424.8886511)
Риск убытков: 0.60

Регион 3:
Средняя прибыль: 411076672.56
95% доверительный интервал: (-138944979.6146276, 959378918.3216925)
Риск убытков: 7.20



Учитывая задачу отобрать регион, для которого риск убытков менее 2.5%, то регион 1 и регион 3 не подходят.

***Вывод:*** <br>
- Датасеты с данными по регионам изучили, проверили на дубликаты, нулевые значения, посмотрели корреляцию признаков к целевому признаку
- Обучили модели для каждого региона и для региона 2 значения RMSE оказались лучшими. RMSE: 0.89
- Обнаружили, что средний объем нефти в скважине для безубыточного функционирования: 111 тыс.баррелей, что выше среднего по 500 скважинам любого из регионов.
- Провели подсчеты по лучшим 200 скважинам в каждом регионе и методом бутстреп нашли 95% доверительный интервал, среднюю прибыль и риск убытков.
- Единственный регион, который удовлетворял нашему заданному риску убытков в 2.5% - это второй. 